In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE batch16v1_bronze
COMMENT "Raw data ingested from Volume batch16v1"
AS
SELECT
  `Sales Person` AS sales_person,
  Country,
  Product,
  Date,
  Amount,
  `Boxes Shipped` AS boxes_shipped
FROM cloud_files(
      "/Volumes/sivaadbuc/default/batch16v1/",
      "csv",
      map("header", "true")
);


In [0]:
CREATE OR REFRESH LIVE TABLE batch16v1_silver
COMMENT "Cleansed data with proper types and filters"
AS
SELECT
  sales_person,
  Country,
  Product,
  TO_DATE(Date, "dd-MMM-yy") AS order_date,
  REPLACE(Amount, '$', '')::DOUBLE AS amount,
  CAST(boxes_shipped AS INT) AS boxes_shipped
FROM LIVE.batch16v1_bronze
WHERE amount IS NOT NULL;


In [0]:
CREATE OR REFRESH LIVE TABLE batch16v1_gold
COMMENT "Aggregated sales and shipments by country and year"
AS
SELECT
  Country,
  YEAR(order_date) AS year,
  SUM(amount) AS total_sales,
  SUM(boxes_shipped) AS total_boxes
FROM LIVE.batch16v1_silver
GROUP BY Country, YEAR(order_date);
